In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import igraph as ig


from random_network import *
from bba import *
from bba_homophily import *
from dpah import *
import itertools
from data_loader import *
from fast_pagerank import pagerank_power
import random
from collections import Counter
import pickle
from numpy import inf

import os
import powerlaw
import numpy as np
import pandas as pd
import networkx as nx
from joblib import delayed
from joblib import Parallel
from collections import Counter
from fast_pagerank import pagerank_power

import joblib
joblib.cpu_count()

### Analysis

In [3]:
BIGNET = 11000
EXT = '.gpickle'
TOPK = 20

#### functions

In [31]:
def _ppr(node_index, A, p, top):
    pp = np.zeros(A.shape[0])
    pp[node_index] = A.shape[0]
    pr = pagerank_power(A, p=p, personalize=pp)
    pr = pr.argsort()[-top-1:][::-1]
    #time.sleep(0.01)
    return pr[pr!=node_index][:top]

def get_circle_of_trust_per_node(A, p=0.85, top=TOPK, num_cores=40):
    return Parallel(n_jobs=num_cores)(delayed(_ppr)(node_index, A, p, top) for node_index in np.arange(A.shape[0]))

def frequency_by_circle_of_trust(A, cot_per_node=None, p=0.85, top=TOPK, num_cores=-1):
    results = cot_per_node if cot_per_node is not None else get_circle_of_trust_per_node(A, p, top, -1)
    unique_elements, counts_elements = np.unique(np.concatenate(results), return_counts=True)
    del(results)
    return [ 0 if node_index not in unique_elements else counts_elements[np.argwhere(unique_elements == node_index)[0, 0]] for node_index in np.arange(A.shape[0])]

def _salsa(node_index, cot, A, top=TOPK):
    BG = nx.Graph()
    BG.add_nodes_from(['h{}'.format(vi) for vi in cot], bipartite=0)  # hubs
    edges = [('h{}'.format(vi), int(vj)) for vi in cot for vj in np.argwhere(A[vi,:] != 0 )[:,1]]
    BG.add_nodes_from(set([e[1] for e in edges]), bipartite=1)  # authorities
    BG.add_edges_from(edges)
    centrality = Counter({n: c for n, c in nx.eigenvector_centrality_numpy(BG).items() if type(n) == int
                                                                                       and n not in cot
                                                                                       and n != node_index
                                                                                       and n not in np.argwhere(A[node_index,:] != 0 )[:,1] })
    del(BG)
    #time.sleep(0.01)
    return np.asarray([n for n, pev in centrality.most_common(top)])[:top]

def get_topj_bc_nodes(network_obj, j=5, to_ig=True, mode='min'):

    def nx_to_ig(G):
        return ig.Graph.from_networkx(G)

    if to_ig:
        g = nx_to_ig(network_obj)
    else:
        g = network_obj

    min_nodes_ig = g.vs.select(m=1)
    maj_nodes_ig = g.vs.select(m=0)

    bc_full = g.betweenness(directed=True)
    g.vs['bc'] = bc_full

    if mode == 'min':
        min_bc = min_nodes_ig.get_attribute_values('bc')
        # maj_bc = min_nodes_ig.get_attribute_values('bc')
        min_bc_temp = min_nodes_ig.get_attribute_values('bc')

        min_bc_temp.sort()
        topj = min_bc_temp[-j:]

        inds = [min_bc.index(t) for t in topj][-j:]
        
    if mode == 'maj':
        maj_bc = maj_nodes_ig.get_attribute_values('bc')
        maj_bc_temp = maj_nodes_ig.get_attribute_values('bc')

        maj_bc_temp.sort()
        topj = maj_bc_temp[-j:]

        inds = [maj_bc.index(t) for t in topj][-j:]

    return inds

def get_topj_evc_nodes(network_obj, j=5, to_ig=True, mode='min'):

    def nx_to_ig(G):
        return ig.Graph.from_networkx(G)

    if to_ig:
        g = nx_to_ig(network_obj)
    else:
        g = network_obj

    min_nodes_ig = g.vs.select(m=1)
    maj_nodes_ig = g.vs.select(m=0)

    evc_full = g.eigenvector_centrality()
    g.vs['evc'] = evc_full

    if mode == 'min':
        min_evc = min_nodes_ig.get_attribute_values('evc')
        # maj_bc = min_nodes_ig.get_attribute_values('bc')
        min_evc_temp = min_nodes_ig.get_attribute_values('evc')

        min_evc_temp.sort()
        topj = min_evc_temp[-j:]

        inds = [min_evc.index(t) for t in topj][-j:]
    if mode == 'maj':
        maj_evc = maj_nodes_ig.get_attribute_values('evc')
        maj_evc_temp = maj_nodes_ig.get_attribute_values('evc')

        maj_evc_temp.sort()
        topj = maj_evc_temp[-j:]

        inds = [maj_evc.index(t) for t in topj][-j:]

    return inds

def DEPR_salsa_intervention(node_index, cot, A, top=TOPK, intervene=False, to_add=None, network=None, strat='bc', metric_mode='min', j=5, target_mode='min'):
        
    BG = nx.Graph()
    BG.add_nodes_from(['h{}'.format(vi) for vi in cot], bipartite=0)  # hubs
    edges = [('h{}'.format(vi), int(vj)) for vi in cot for vj in np.argwhere(A[vi,:] != 0 )[:,1]]
    BG.add_nodes_from(set([e[1] for e in edges]), bipartite=1)  # authorities
    BG.add_edges_from(edges)
    centrality = Counter({n: c for n, c in nx.eigenvector_centrality_numpy(BG).items() if type(n) == int
                                                                                       and n not in cot
                                                                                       and n != node_index
                                                                                       and n not in np.argwhere(A[node_index,:] != 0 )[:,1] })
    del(BG)
    
    return np.asarray([n for n, pev in centrality.most_common(top)])[:top]

def _salsa2(node_index, cot, A, top=TOPK):
    
    BG = nx.Graph()
    BG.add_nodes_from(['h{}'.format(vi) for vi in cot], bipartite=0)  # hubs
    edges = [('h{}'.format(vi), int(vj)) for vi in cot for vj in np.argwhere(A[vi,:] != 0 )[:,1]]
    BG.add_nodes_from(set([e[1] for e in edges]), bipartite=1)  # authorities
    BG.add_edges_from(edges)
    centrality = Counter({n: c for n, c in nx.eigenvector_centrality_numpy(BG).items() if type(n) == int
                                                                                       and n not in cot
                                                                                       and n != node_index
                                                                                       and n not in np.argwhere(A[node_index,:] != 0 )[:,1] })

    return np.asarray([n for n, pev in centrality.most_common(top)])[:top]

def _get_hubs_auths(node_index, cot, A, top=TOPK):
    
    BG = nx.Graph()
    BG.add_nodes_from(['h{}'.format(vi) for vi in cot], bipartite=0)  # hubs
    edges = [('h{}'.format(vi), int(vj)) for vi in cot for vj in np.argwhere(A[vi,:] != 0 )[:,1]]
    BG.add_nodes_from(set([e[1] for e in edges]), bipartite=1)  # authorities
    BG.add_edges_from(edges)

    auths = [node for node, data in BG.nodes(data=True) if data['bipartite'] == 1]
    hubs = [int(node[1:]) for node, data in BG.nodes(data=True) if data['bipartite'] == 0]

    return {node_index: {'hubs': hubs, 'auths': auths}}

def frequency_by_who_to_follow(A, cot_per_node=None, p=0.85, top=TOPK, num_cores=40):
    cot_per_node = cot_per_node if cot_per_node is not None else get_circle_of_trust_per_node(A, p, top, -1)
    results = Parallel(n_jobs=-1)(delayed(_salsa)(node_index, cot, A, top) for node_index, cot in enumerate(cot_per_node))
    unique_elements, counts_elements = np.unique(np.concatenate(results), return_counts=True)
    del(results)
    return [0 if node_index not in unique_elements else counts_elements[np.argwhere(unique_elements == node_index)[0, 0]] for node_index in np.arange(A.shape[0])]

def frequency_by_who_to_follow2(A, cot_per_node=None, p=0.85, top=TOPK, num_cores=-1):

    cot_per_node = cot_per_node if cot_per_node is not None else get_circle_of_trust_per_node(A, p, top, num_cores)
    results = Parallel(n_jobs=-1)(delayed(_salsa2)(node_index, cot, A, top) for node_index, cot in enumerate(cot_per_node))

    return results

def frequency_by_who_to_follow3(A, cot_per_node=None, p=0.85, top=TOPK, num_cores=-1):

    cot_per_node = cot_per_node if cot_per_node is not None else get_circle_of_trust_per_node(A, p, top, num_cores)
    ha_dict = Parallel(n_jobs=-1)(delayed(_get_hubs_auths)(node_index, cot, A, top) for node_index, cot in enumerate(cot_per_node))

    return ha_dict

def who_to_follow_rank(A, njobs=-1):
    if A.shape[0] < BIGNET:
        return wtf_small(A=A, njobs=njobs)
    else:
        return wtf_small(A=A, njobs=njobs)
        
def wtf_small(A, njobs):
    # utils.printf('cot_per_node...')
    cot_per_node = get_circle_of_trust_per_node(A, p=0.85, top=TOPK, num_cores=njobs)

    # utils.printf('cot...')
    # cot = frequency_by_circle_of_trust(A, cot_per_node=cot_per_node, p=0.85, top=TOPK, num_cores=njobs)

    # utils.printf('wtf...')
    wtf = frequency_by_who_to_follow3(A=A, cot_per_node=cot_per_node, p=0.85, top=TOPK, num_cores=njobs)
    return wtf

In [5]:
def twitter_wtf(network, sparse_adj, node_id, k_for_circle_of_trust=20, cot_factor= 0.05, tol=1e-8,
                damping_factor=.85, k_for_recommendation=-1):
    """This method aims to realize a link prediction algorithm used by Twitter to perform
        the WTF recommendation on the platform.
        The algorithm can be seen at 'https://web.stanford.edu/~rezab/papers/wtf_overview.pdf'.
        The algorithm consists of two phases:
            1) Compute the circle of trust for the user you want to recommend(top-k nodes in PPR)
            2) Compute the top-k nodes using score propagation
    """
    k_for_circle_of_trust = min(int(network.number_of_nodes()*cot_factor), k_for_circle_of_trust)
    #1st phase: Compute circle of trust of user according to Personalized PageRank
    personalize = np.zeros(shape=network.number_of_nodes())
    personalize[node_id] = 1
    values_of_personalized_pr = pagerank_power(sparse_adj, p=damping_factor, personalize=personalize, tol=1e-6)
    circle_of_trust = values_of_personalized_pr.argsort()[-k_for_circle_of_trust:][::-1]

    #2nd phase: init bipartite graph
    bipartite_graph = nx.DiGraph()
    #add nodes belonging to the circle of trust as hubs(H)
    for node in circle_of_trust:
        #these nodes are "hubs"(H) in the bipartite graph
        bipartite_graph.add_node(str(node)+"H")
    #add out neighbors of nodes belonging to the circle of trust as authorities(A)
    for node in circle_of_trust:
        for out_neighbor in network.neighbors(node):
            #direction is inverted for a matter of simplicity in the sequent phases
            bipartite_graph.add_edge(str(out_neighbor)+"A", str(node)+"H")

    #retrieve adjacency matrix of bipartite graph
    A = nx.to_numpy_array(bipartite_graph)

    #retrieve list of all nodes splitted by authority or hub
    all_nodes = list(bipartite_graph.nodes())
    hub_nodes = [int(x[:-1]) for x in all_nodes if 'H' in x]
    authority_nodes = [int(x[:-1]) for x in all_nodes if 'A' in x]

    #3rd phase: start building ingredients of our SALSA algorithm
    #these are the transition matrices determined by the bipartite graph
    S_prime = A[len(hub_nodes):, :][:, :len(hub_nodes)].copy()
    R_prime = S_prime.T.copy()
    #normalize both matrices
    denominator_S_prime = S_prime.sum(axis=0)
    denominator_S_prime[denominator_S_prime == 0] = 1
    S_prime = S_prime / denominator_S_prime
    denominator_R_prime = R_prime.sum(axis=0)
    denominator_R_prime[denominator_R_prime == 0] = 1
    R_prime = R_prime / denominator_R_prime
    #these are the vectors which contain the score of similarity
    #and relevance
    s = np.zeros(shape=(len(hub_nodes), 1), dtype=np.float)
    r = np.zeros(shape=(len(authority_nodes), 1), dtype=np.float)

    #at the beginning of the procedure we put the similarity
    #of the user we want to give the recommendation equal to 1
    index_of_node_to_recommend = np.where(circle_of_trust == node_id)[0][0]
    s[index_of_node_to_recommend] = 1.

    #init damping vector
    alpha = 1 - damping_factor
    alpha_vector = np.zeros(shape=(len(hub_nodes), 1), dtype=np.float)
    alpha_vector[index_of_node_to_recommend] = alpha

    #4th phase: run the algorithm
    convergence = False
    while not convergence:
        s_ = s.copy()
        r_ = r.copy()
        r_ = S_prime.dot(s)
        s_ = alpha_vector + (1 - alpha)*(R_prime.dot(r))
        #compute difference and check if convergence has been reached
        diff = abs(s_ - s)
        if np.linalg.norm(diff) < tol:
            convergence=True
        #update real vectors
        s = s_
        r = r_

    #5th phase: order by score and delete neighbors of node to be recommended
    #of course we don't want to recommend people that the user already follow
    neighbors_to_not_recommend = nx.neighbors(network, node_id)
    relevance_scores = r.flatten()
    if k_for_recommendation == -1:
        k_for_recommendation = 0 #Take all the nodes!

    neighbors_to_not_recommend = set(neighbors_to_not_recommend)
    results = []
    for node in relevance_scores.argsort()[::-1]:
        if node not in neighbors_to_not_recommend and node != node_id:
            results.append(((node_id, node))) # , relevance_scores[node]
            if len(results) == k_for_recommendation:
                break
    return results

In [6]:
def get_topj_bc_nodes(network_obj, j=5, to_ig=True, mode='min'):

    def nx_to_ig(G):
        return ig.Graph.from_networkx(G)

    if to_ig:
        g = nx_to_ig(network_obj)
    else:
        g = network_obj

    min_nodes_ig = g.vs.select(m=1)
    maj_nodes_ig = g.vs.select(m=0)

    bc_full = g.betweenness(directed=True)
    g.vs['bc'] = bc_full

    if mode == 'min':
        min_bc = min_nodes_ig.get_attribute_values('bc')
        # maj_bc = min_nodes_ig.get_attribute_values('bc')
        min_bc_temp = min_nodes_ig.get_attribute_values('bc')

        min_bc_temp.sort()
        topj = min_bc_temp[-j:]

        inds = [bc_full.index(t) for t in topj][-j:]

        return inds

    if mode == 'maj':
        maj_bc = maj_nodes_ig.get_attribute_values('bc')
        maj_bc_temp = maj_nodes_ig.get_attribute_values('bc')

        maj_bc_temp.sort()
        topj = maj_bc_temp[-j:]

        inds = [bc_full.index(t) for t in topj][-j:]

        return inds
    
    elif mode == 'both':
        min_bc = min_nodes_ig.get_attribute_values('bc')
        min_bc_temp = min_nodes_ig.get_attribute_values('bc')

        min_bc_temp.sort()
        topj = min_bc_temp[-j:]

        inds_min = [bc_full.index(t) for t in topj][-j:]

        maj_bc = maj_nodes_ig.get_attribute_values('bc')
        maj_bc_temp = maj_nodes_ig.get_attribute_values('bc')

        maj_bc_temp.sort()
        topj = maj_bc_temp[-j:]

        inds_maj = [bc_full.index(t) for t in topj][-j:]

        return inds_maj, inds_min


def get_topj_evc_nodes(network_obj, j=5, to_ig=True, mode='min'):

    def nx_to_ig(G):
        return ig.Graph.from_networkx(G)

    if to_ig:
        g = nx_to_ig(network_obj)
    else:
        g = network_obj

    min_nodes_ig = g.vs.select(m=1)
    maj_nodes_ig = g.vs.select(m=0)

    evc_full = g.eigenvector_centrality()
    g.vs['evc'] = evc_full

    if mode == 'min':
        min_evc = min_nodes_ig.get_attribute_values('evc')
        min_evc_temp = min_nodes_ig.get_attribute_values('evc')

        min_evc_temp.sort()
        topj = min_evc_temp[-j:]

        inds = [evc_full.index(t) for t in topj][-j:]

        return inds
    
    elif mode == 'maj':
        maj_evc = maj_nodes_ig.get_attribute_values('evc')
        maj_evc_temp = maj_nodes_ig.get_attribute_values('evc')

        maj_evc_temp.sort()
        topj = maj_evc_temp[-j:]

        inds = [evc_full.index(t) for t in topj][-j:]

        return inds

    elif mode == 'both':
        min_evc = min_nodes_ig.get_attribute_values('evc')
        min_evc_temp = min_nodes_ig.get_attribute_values('evc')

        min_evc_temp.sort()
        topj = min_evc_temp[-j:]

        inds_min = [evc_full.index(t) for t in topj][-j:]

        maj_evc = maj_nodes_ig.get_attribute_values('evc')
        maj_evc_temp = maj_nodes_ig.get_attribute_values('evc')

        maj_evc_temp.sort()
        topj = maj_evc_temp[-j:]

        inds_maj = [evc_full.index(t) for t in topj][-j:]

        return inds_maj, inds_min


def _ppr(node_index, A, p, top):
    pp = np.zeros(A.shape[0])
    pp[node_index] = A.shape[0]
    pr = pagerank_power(A, p=p, personalize=pp)
    pr = pr.argsort()[-top-1:][::-1]
    #time.sleep(0.01)
    return list(pr[pr!=node_index][:top])

def _get_circle_of_trust_per_node(A, p=0.85, top=10, num_cores=-1):
    return Parallel(n_jobs=num_cores)(delayed(_ppr)(node_index, A, p, top) for node_index in np.arange(A.shape[0]))

def _salsa_intervention(node_index, cot, A, top=10, to_add=[], network_obj=None):

    BG = nx.Graph()
    BG.add_nodes_from(['h{}'.format(vi) for vi in cot], bipartite=0)  # hubs
    edges = [('h{}'.format(vi), int(vj)) for vi in cot for vj in np.argwhere(A[vi,:] != 0 )[:,1]]
    BG.add_nodes_from(set([e[1] for e in edges]), bipartite=1)  # authorities
    BG.add_edges_from(edges)

    auths = [node for node, data in BG.nodes(data=True) if data['bipartite'] == 1]
    hubs = [int(node[1:]) for node, data in BG.nodes(data=True) if data['bipartite'] == 0]

    if network_obj.nodes()[node_index]['m'] == 1:
        to_add = to_add[1]
    else:
        to_add = to_add[0]

        new_auths = [auth_node for auth_node in to_add if auth_node not in auths]
        new_auths_not_hubs = [n for n in new_auths if n not in hubs]

        intervention_edges = [(hub, new_auth) for hub in random.sample(hubs, int(len(hubs) * 0.15)) for new_auth in new_auths_not_hubs]
        BG.add_nodes_from(new_auths_not_hubs, bipartite=1)
        BG.add_edges_from(intervention_edges)

    centrality = Counter({n: c for n, c in nx.eigenvector_centrality_numpy(BG).items() if type(n) == int
                                                                                        and n not in cot
                                                                                        and n != node_index
                                                                                        and n not in np.argwhere(A[node_index,:] != 0 )[:,1] })

    return np.asarray([n for n, pev in centrality.most_common(top)])[:top]

def get_intervention_recommendations(A, to_add=[], p=0.85, top=10, num_cores=-1, network_obj=None):
    
    cot_per_node = _get_circle_of_trust_per_node(A, p, top, num_cores)
    results = Parallel(n_jobs=num_cores)(delayed(_salsa_intervention)(node_index, cot, A, top, to_add, network_obj) for node_index, cot in enumerate(cot_per_node))
    
    return results

In [22]:
class Analyzer:

    def __init__(self, network, class_name='m', min_label=1, maj_label=0):
        self.network = network
        self.min_label, self.maj_label = min_label, maj_label
        # self.min_nodes, self.maj_nodes = self.get_nodes_per_class()
        # self.post_network = None
        self.class_name = class_name
        self.metrics_dict = {}
    
    def get_nodes_per_class(self, network):
        
        # class_
        if self.class_name == 'm':
            min_nodes_ig = network.vs.select(m=self.min_label)
            maj_nodes_ig = network.vs.select(m=self.maj_label)

        elif self.class_name == 'class_':
            min_nodes_ig = network.vs.select(class_=self.min_label)
            maj_nodes_ig = network.vs.select(class_=self.maj_label)
        
        elif self.class_name == 'age':
            min_nodes_ig = network.vs.select(age=self.min_label)
            maj_nodes_ig = network.vs.select(age=self.maj_label)

        return min_nodes_ig, maj_nodes_ig
    
    def get_group_betweenness(self, network, is_pre=True, verbose=True):

        min_nodes, maj_nodes = self.get_nodes_per_class(network)

        bc_full = network.betweenness(directed=True)
        network.vs['bc'] = bc_full

        min_bc = np.average(np.array(min_nodes.get_attribute_values('bc')))
        maj_bc = np.average(np.array(maj_nodes.get_attribute_values('bc')))
        
        if is_pre:
            self.metrics_dict['bc'] = {'pre': {}, 'post': {}}
            self.metrics_dict['bc']['pre'] = {'min': min_bc, 'maj': maj_bc}
        else:
            self.metrics_dict['bc']['post'] = {'min': min_bc, 'maj': maj_bc}

        if verbose:
            print(f"Minority BC: {min_bc}")
            print(f"Majority BC: {maj_bc}")

        return min_bc, maj_bc
    
    def get_group_closeness(self, network, is_pre=True, verbose=True):

        min_nodes, maj_nodes = self.get_nodes_per_class(network)

        cc_full = network.closeness()
        network.vs['cc'] = cc_full

        min_cc = np.average(np.array(min_nodes.get_attribute_values('cc')))
        maj_cc = np.average(np.array(maj_nodes.get_attribute_values('cc')))

        if is_pre:
            self.metrics_dict['cc'] = {'pre': {}, 'post': {}}
            self.metrics_dict['cc']['pre'] = {'min': min_cc, 'maj': maj_cc}
        else:
            self.metrics_dict['cc']['post'] = {'min': min_cc, 'maj': maj_cc}

        if verbose:
            print(f"Minority CC: {min_cc}")
            print(f"Majority CC: {maj_cc}")

        return min_cc, maj_cc
    
    def get_group_eigenvector_centrality(self, network, is_pre=True, verbose=True):

        min_nodes, maj_nodes = self.get_nodes_per_class(network)

        evc_full = network.eigenvector_centrality()
        network.vs['evc'] = evc_full

        min_evc = np.average(np.array(min_nodes.get_attribute_values('evc')))
        maj_evc = np.average(np.array(maj_nodes.get_attribute_values('evc')))
        
        if is_pre:
            self.metrics_dict['evc'] = {'pre': {}, 'post': {}}
            self.metrics_dict['evc']['pre'] = {'min': min_evc, 'maj': maj_evc}
        else:
            self.metrics_dict['evc']['post'] = {'min': min_evc, 'maj': maj_evc}

        if verbose:
            print(f"Minority EVC: {min_evc} / normalized: {min_evc/(min_evc+maj_evc)}")
            print(f"Majority EVC: {maj_evc} / normalized: {maj_evc/(min_evc+maj_evc)}")

        return min_evc, maj_evc
    
    def get_group_degree_centrality(self, network, is_pre=True, verbose=True):

        min_nodes, maj_nodes = self.get_nodes_per_class(network)

        dc_full = network.vs.indegree()
        network.vs['dc'] = dc_full

        min_dc = np.average(np.array(min_nodes.get_attribute_values('dc')))
        maj_dc = np.average(np.array(maj_nodes.get_attribute_values('dc')))
        
        if is_pre:
            self.metrics_dict['dc'] = {'pre': {}, 'post': {}}
            self.metrics_dict['dc']['pre'] = {'min': min_dc, 'maj': maj_dc}
        else:
            self.metrics_dict['dc']['post'] = {'min': min_dc, 'maj': maj_dc}

        if verbose:
            print(f"Minority DC: {min_dc}")
            print(f"Majority DC: {maj_dc}")

        return min_dc, maj_dc

    def get_group_pagerank(self, network, is_pre=True, verbose=True):

        min_nodes, maj_nodes = self.get_nodes_per_class(network)
        pr_full = network.pagerank()

        network.vs['pr'] = pr_full

        min_pr = np.average(np.array(min_nodes.get_attribute_values('pr')))
        maj_pr = np.average(np.array(maj_nodes.get_attribute_values('pr')))
        
        if is_pre:
            self.metrics_dict['pr'] = {'pre': {}, 'post': {}}
            self.metrics_dict['pr']['pre'] = {'min': min_pr, 'maj': maj_pr}
        else:
            self.metrics_dict['pr']['post'] = {'min': min_pr, 'maj': maj_pr}

        if verbose:
            print(f"Minority PR: {min_pr}")
            print(f"Majority PR: {maj_pr}")

        return min_pr, maj_pr
    
    def remove_zero_degree(self, verbose=False):
        self.to_remove = [node for node, degree in dict(self.network.degree()).items() if degree < 1]
        self.network.remove_nodes_from(self.to_remove)

        if verbose:
            print(f"Removed {len(self.to_remove)} nodes with degree zero.")
        return self.to_remove

    def add_nodes(self):
        self.network.add_nodes_from(self.to_remove)
        # print(f"Added {len(self.to_remove)} nodes.")

    def sample_and_delete_edges(self, num_to_sample=100, remove_edges=True):
        edge_list = list(self.network.edges())

        self.random_edges_to_delete = random.sample(edge_list, num_to_sample)
        nodes_with_del_out_edge = [tup[0] for tup in self.random_edges_to_delete]
        self.unique_nodes = list(set(nodes_with_del_out_edge))

        self.nodes_with_edge_count = Counter(node for node in nodes_with_del_out_edge)

        if remove_edges:
            self.network.remove_edges_from(self.random_edges_to_delete)

            # print(f'# nodes with deleted out edge {len(nodes_with_del_out_edge)}')
            print(f'# unique nodes with deleted out edge: {len(self.unique_nodes)}')
        else:
            print(f'# unique nodes with sampled out edge: {len(self.unique_nodes)} - No edges where deleted.')

        return self.random_edges_to_delete, self.unique_nodes, self.nodes_with_edge_count

    def reconstruct_edges(self, strategy='bulk', k_for_circle_of_trust=20, salsa_new=True, run=0, wtf=None, to_pick=1,
                            consider_all_nodes=True, num_to_consider=1000, add_top_k=False, delete_edges=True, 
                            delete_first=False, network_file='', network_obj=None, class_name='m',
                            intervene=False, to_add=None, network=None, strat='bc', metric_mode='min', j=5, target_mode='min'):


        if strategy == 'intervention':

            self.salsa_edges_per_node = {}
            self.salsa_all_edges = []
            self.top_k_frac_min = []
            self.edge_type = {'mm': 0,
                            'MM': 0,
                            'mM': 0,
                            'Mm': 0}

            all_nodes = list(self.network.nodes())
            sample = random.sample(all_nodes, num_to_consider)
            evc = nx.eigenvector_centrality(self.network)

            for node in sample:

                wtf_node_i = list(wtf[node])

                if len(wtf_node_i) >= to_pick:
                    self.salsa_edges_per_node[node] = []

                    sub_dict = {i: evc[i] for i in wtf_node_i}
                    sub_dict_sorted = dict(sorted(sub_dict.items(), key=lambda item: item[1], reverse=True))
                    pick = list(sub_dict_sorted.keys())[:to_pick]


                    if delete_edges:

                        out_edges_node = list(self.network.out_edges(node))

                        # num_to_remove = min(len(out_edges_node), to_pick)

                        # self.network.remove_edges_from(random.sample(out_edges_node, num_to_remove))

                        if len(out_edges_node) >= to_pick:
                            self.network.remove_edges_from(random.sample(out_edges_node, to_pick))

                            for new_node in pick:
                                self.network.add_edge(node, new_node)
                                self.salsa_all_edges.append((node, new_node))
                            self.salsa_edges_per_node[node].append(pick)   

                            top_k_classes = [self.network.nodes()[node][self.class_name] for node in wtf_node_i]
                            frac_min = sum(top_k_classes) / len(top_k_classes)
                            frac_maj = 1-frac_min
                            self.top_k_frac_min.append(frac_min*100)                                


        if strategy == 'new':

            self.salsa_edges_per_node = {}
            self.salsa_all_edges = []
            self.top_k_frac_min = []
            self.edge_type = {'mm': 0,
                            'MM': 0,
                            'mM': 0,
                            'Mm': 0}


            top = 10
            p = 0.85

            sample = random.sample(self.network.nodes(), num_to_consider)


            for k in range(to_pick):
                
                A = nx.adjacency_matrix(self.network)

                cots = Parallel(n_jobs=-1)(delayed(_ppr)(node, A, p, top) for node in sample)
                z = dict(zip(sample, cots))
                recommendations = Parallel(n_jobs=-1)(delayed(_salsa2)(node, z[node], A, ix=None, top=10) for node in z.keys())

                for arr in recommendations:

                    if network_file == '':
                            network = network_obj
                    else:
                            network = read_pkl_graph(network_file)
                    
                    top_k_classes = [network.nodes()[node][class_name] for node in arr]
                    frac_min = sum(top_k_classes) / len(top_k_classes)
                    frac_maj = 1-frac_min
                    self.top_k_frac_min.append(frac_min*100)

                zz = dict(zip(sample, recommendations))
                for key in zz.keys():
                    to_add = zz[key][0]
                    self.network.add_edge(key, to_add)
                    self.salsa_all_edges.append((key, to_add))
                    self.network.remove_edges_from(random.sample(list(self.network.out_edges(key)), 1))    


        if strategy == 'consecutively':
            self.salsa_edges_per_node = {}
            self.salsa_all_edges = []
            self.top_k_frac_min = []
            self.edge_type = {'mm': 0,
                            'MM': 0,
                            'mM': 0,
                            'Mm': 0}
            num_edges_added = 0

            if salsa_new:

                wtf = wtf

                if consider_all_nodes:
                
                    for node in self.network.nodes():
                        # num_to_add = self.nodes_with_edge_count[node]

                        wtf_node_i = list(wtf[node])
                        if len(wtf_node_i) >= to_pick:
                            self.salsa_edges_per_node[node] = []

                            if add_top_k:
                                pick = wtf_node_i[0:to_pick]
                                
                            else:
                                pick = random.sample(wtf_node_i, to_pick)

                            
                            if delete_edges:
                                out_edges_node = list(self.network.out_edges(node))
                               
                                if len(out_edges_node) >= to_pick:
                                    self.network.remove_edges_from(random.sample(out_edges_node, to_pick))

                                    for new_node in pick:
                                        self.network.add_edge(node, new_node)
                                        self.salsa_all_edges.append((node, new_node))
                                    self.salsa_edges_per_node[node].append(pick)

                                    top_k_classes = [self.network.nodes()[node][self.class_name] for node in wtf_node_i]
                                    frac_min = sum(top_k_classes) / len(top_k_classes)
                                    frac_maj = 1-frac_min
                                    self.top_k_frac_min.append(frac_min*100)                                
                                    
                            else:
                                for new_node in pick:
                                        self.network.add_edge(node, new_node)
                                        self.salsa_all_edges.append((node, new_node))
                                self.salsa_edges_per_node[node].append(pick)

                                top_k_classes = [self.network.nodes()[node][self.class_name] for node in wtf_node_i]
                                frac_min = sum(top_k_classes) / len(top_k_classes)
                                frac_maj = 1-frac_min
                                self.top_k_frac_min.append(frac_min*100)                                
                            
                                num_edges_added += 1
                
                else:
                    
                    all_nodes = list(self.network.nodes())
                    sample = random.sample(all_nodes, num_to_consider)

                    for node in sample:

                        wtf_node_i = list(wtf[node])


                        if len(wtf_node_i) >= to_pick:
                            self.salsa_edges_per_node[node] = []

                            if add_top_k:
                                pick = wtf_node_i[0:to_pick]
                            else:
                                pick = random.sample(wtf_node_i, to_pick)

                            if delete_edges:

                                out_edges_node = list(self.network.out_edges(node))

                                # num_to_remove = min(len(out_edges_node), to_pick)

                                # self.network.remove_edges_from(random.sample(out_edges_node, num_to_remove))

                                # for new_node in pick:
                                #     self.network.add_edge(node, new_node)
                                #     self.salsa_all_edges.append((node, new_node))
                                # self.salsa_edges_per_node[node].append(pick)   

                                # top_k_classes = [self.network.nodes()[node][self.class_name] for node in wtf_node_i]
                                # frac_min = sum(top_k_classes) / len(top_k_classes)
                                # frac_maj = 1-frac_min
                                # self.top_k_frac_min.append(frac_min*100)    


                                if len(out_edges_node) >= to_pick:
                                    self.network.remove_edges_from(random.sample(out_edges_node, to_pick))

                                    for new_node in pick:
                                        self.network.add_edge(node, new_node)
                                        self.salsa_all_edges.append((node, new_node))
                                    self.salsa_edges_per_node[node].append(pick)   

                                    top_k_classes = [self.network.nodes()[node][self.class_name] for node in wtf_node_i]
                                    frac_min = sum(top_k_classes) / len(top_k_classes)
                                    frac_maj = 1-frac_min
                                    self.top_k_frac_min.append(frac_min*100)                                

                            else:

                                for new_node in pick:
                                        self.network.add_edge(node, new_node)
                                        self.salsa_all_edges.append((node, new_node))
                                self.salsa_edges_per_node[node].append(pick)   

                                top_k_classes = [self.network.nodes()[node][self.class_name] for node in wtf_node_i]
                                frac_min = sum(top_k_classes) / len(top_k_classes)
                                frac_maj = 1-frac_min
                                self.top_k_frac_min.append(frac_min*100) 
                                
            
            else: 

                for node in self.unique_nodes:
                    self.salsa_edges_per_node[node] = []
                    for k in range(self.nodes_with_edge_count[node]):

                        top_all = twitter_wtf(network=self.network, sparse_adj=nx.adjacency_matrix(self.network), node_id=node, k_for_recommendation=-1)
                        top_1 = top_all[0]
                        self.network.add_edge(node, top_1[1])
                        
                        top_10pct = top_all[:int(len(top_all)*0.1)]
                        
                        # top_10pct_classes = [self.network.nodes()[edge[1]][self.class_name] for edge in top_10pct]
                        top_10pct_classes = [self.network.nodes()[edge[-1]][self.class_name] for edge in top_10pct]
                        frac_min = sum(top_10pct_classes) / len(top_10pct_classes)
                        frac_maj = 1-frac_min
                        self.top_10pct_frac_min.append(frac_min)

                        self.salsa_edges_per_node[node].append(top_1)
                        self.salsa_all_edges.append(top_1)
                        
                        num_edges_added += 1   
        
        elif strategy == 'random':

            if consider_all_nodes:

                self.salsa_edges_per_node = {}
                self.salsa_all_edges = []
                self.top_k_frac_min = []
                self.edge_type = {'mm': 0,
                            'MM': 0,
                            'mM': 0,
                            'Mm': 0}
                
                self.all_nodes = list(self.network.nodes())
                for node in self.all_nodes:
                    
                    pick = random.sample(self.all_nodes, to_pick)

                    if delete_edges:
                        out_edges_node = list(self.network.out_edges(node))

                        if len(out_edges_node) >= to_pick:
                            self.network.remove_edges_from(random.sample(out_edges_node, to_pick))

                            for new_node in pick:
                                self.network.add_edge(node, new_node)
                                self.salsa_all_edges.append((node, new_node))
                            self.salsa_edges_per_node[node] = pick
                    else:
                        for new_node in pick:
                            self.network.add_edge(node, new_node)
                            self.salsa_all_edges.append((node, new_node))
                        self.salsa_edges_per_node[node] = pick
            
            else:
                self.salsa_edges_per_node = {}
                self.salsa_all_edges = []
                self.top_k_frac_min = []
                self.edge_type = {'mm': 0,
                            'MM': 0,
                            'mM': 0,
                            'Mm': 0}
                
                self.all_nodes = list(self.network.nodes())
                k = random.sample(self.all_nodes, num_to_consider)

                for node in k:
                    
                    pick = random.sample(k, to_pick)

                    if delete_edges:
                        out_edges_node = list(self.network.out_edges(node))

                        if len(out_edges_node) >= to_pick:
                            self.network.remove_edges_from(random.sample(out_edges_node, to_pick))

                            for new_node in pick:
                                self.network.add_edge(node, new_node)
                                self.salsa_all_edges.append((node, new_node))
                            self.salsa_edges_per_node[node] = pick

                    else:
                        for new_node in pick:
                                self.network.add_edge(node, new_node)
                                self.salsa_all_edges.append((node, new_node))
                        self.salsa_edges_per_node[node] = pick

        self.avg_frac_min = 0

        if strategy != 'random':
            self.avg_frac_min, self.sd_frac_min = np.round(np.average(self.top_k_frac_min), 5), np.round(np.std(self.top_k_frac_min), 5)


        return self.salsa_edges_per_node, self.salsa_all_edges, self.edge_type, self.avg_frac_min

    def add_edges(self, strategy='bulk', k_for_circle_of_trust=20):

        if strategy == 'bulk':
            self.salsa_edges_per_node = {}
            self.salsa_all_edges = []
            num_edges_added = 0
            for node in self.unique_nodes:

                recommended_nodes = twitter_wtf(network=self.network, sparse_adj=nx.adjacency_matrix(self.network), node_id=node, k_for_circle_of_trust=k_for_circle_of_trust, k_for_recommendation=self.nodes_with_edge_count[node])
                self.network.add_edges_from(recommended_nodes)
                self.salsa_edges_per_node[node] = recommended_nodes
                self.salsa_all_edges.append(recommended_nodes[0])
                num_edges_added += len(recommended_nodes)

        elif strategy == 'random':
            self.salsa_edges_per_node = {}
            self.salsa_all_edges = []
            num_edges_added = 0
            self.all_nodes = list(self.network.nodes())
            for node in self.unique_nodes:
                
                for z in range(self.nodes_with_edge_count[node]):
                    pick = random.sample(self.all_nodes, 1)[0]

                    self.network.add_edge(node, pick)
                    self.salsa_edges_per_node[node] = pick
                    self.salsa_all_edges.append(pick)
                    num_edges_added += 1
        
        return self.salsa_edges_per_node, self.salsa_all_edges

    def get_relative_metric_changes(self, metric='all', round=5):

        if metric == 'all':
           
            bc_vals = self.metrics_dict['bc']
            print("Relative BC changes")
            print("---------------------------")
            print(f"Majority change (pre->post) %: {np.round((bc_vals['post']['maj']/bc_vals['pre']['maj']-1)*100, round)}%")
            print(f"Minority change (pre->post) %: {np.round((bc_vals['post']['min']/bc_vals['pre']['min']-1)*100, round)}%")
            print("")
        
            cc_vals = self.metrics_dict['cc']
            print("Relative CC changes")
            print("---------------------------")
            print(f"Majority change (pre->post) %: {np.round((cc_vals['post']['maj']/cc_vals['pre']['maj']-1)*100, round)}%")
            print(f"Minority change (pre->post) %: {np.round((cc_vals['post']['min']/cc_vals['pre']['min']-1)*100, round)}%")
            print("")
        
            evc_vals = self.metrics_dict['evc']
            print("Relative EVC changes")
            print("---------------------------")
            print(f"Majority change (pre->post) %: {np.round((evc_vals['post']['maj']/evc_vals['pre']['maj']-1)*100, round)}%")
            print(f"Minority change (pre->post) %: {np.round((evc_vals['post']['min']/evc_vals['pre']['min']-1)*100, round)}%")
            # print(f"Normalized Majority change (pre->post) %: {np.round(((evc_vals['post']['maj']/(evc_vals['post']['maj']+evc_vals['post']['min']))/(evc_vals['pre']['maj']/(evc_vals['pre']['maj']+evc_vals['pre']['min']))-1)*100)}%")
            # print(f"Normalized Minority change (pre->post) %: {np.round((evc_vals['post']['min']/(evc_vals['pre']['min']+evc_vals['pre']['maj'])-1)*100, round)}%")
            print("")

            dc_vals = self.metrics_dict['dc']
            print("Relative DC changes")
            print("---------------------------")
            print(f"Majority change (pre->post) %: {np.round((dc_vals['post']['maj']/dc_vals['pre']['maj']-1)*100, round)}%")
            print(f"Minority change (pre->post) %: {np.round((dc_vals['post']['min']/dc_vals['pre']['min']-1)*100, round)}%")
            print("")

        elif metric == 'bc':
            bc_vals = self.metrics_dict['bc']

            print("Relative BC changes")
            print("---------------------------")
            print(f"Majority change (pre->post) %: {np.round((bc_vals['post']['maj']/bc_vals['pre']['maj']-1)*100, round)}%")
            print(f"Minority change (pre->post) %: {np.round((bc_vals['post']['min']/bc_vals['pre']['min']-1)*100, round)}%")
        
        elif metric == 'cc':
            cc_vals = self.metrics_dict['cc']
            
            print("Relative CC changes")
            print("---------------------------")
            print(f"Majority change (pre->post) %: {np.round((cc_vals['post']['maj']/cc_vals['pre']['maj']-1)*100, round)}%")
            print(f"Minority change (pre->post) %: {np.round((cc_vals['post']['min']/cc_vals['pre']['min']-1)*100, round)}%")
        
        elif metric == 'evc':
            evc_vals = self.metrics_dict['evc']
            
            print("Relative EVC changes")
            print("---------------------------")
            print(f"Majority change (pre->post) %: {np.round((evc_vals['post']['maj']/evc_vals['pre']['maj']-1)*100, round)}%")
            print(f"Minority change (pre->post) %: {np.round((evc_vals['post']['min']/evc_vals['pre']['min']-1)*100, round)}%")
        
        elif metric == 'dc':
            dc_vals = self.metrics_dict['dc']
            
            print("Relative DC changes")
            print("---------------------------")
            print(f"Majority change (pre->post) %: {np.round((dc_vals['post']['maj']/dc_vals['pre']['maj']-1)*100, round)}%")
            print(f"Minority change (pre->post) %: {np.round((dc_vals['post']['min']/dc_vals['pre']['min']-1)*100, round)}%")

In [23]:
def nx_to_ig(G):
    return ig.Graph.from_networkx(G)

In [24]:
def rename_class_attribute(G, from_='', to_=''):
    class_ = G.vs()[from_]
    G.vs[to_] = class_

In [25]:
def remove_zero_degree(network, verbose=False):
    to_remove = [node for node, degree in dict(network.degree()).items() if degree < 1]
    network.remove_nodes_from(to_remove)

    if verbose:
        print(f"Removed {len(to_remove)} nodes with degree zero.")
    return to_remove

In [11]:
def run_n_simulations(network_obj=None, network_file='', n=5, num_to_sample=10000, delete_edges=True, verbose=False, round=5, 
                      remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                      strategy='bulk', save_results=True, save_as='', to_txt=True, salsa_new=True, quick_test=False,
                      to_pick=1, consider_all_nodes=True, num_to_consider=1000, add_top_k=False,
                      intervene=False, strat='evc', metric_mode='both', j=5, target_mode='min', recomms=None):

    

    changes_dict_min = {'bc': [],
                        'cc': [],
                        'evc_norm': [],
                        'evc': [],
                        'evc_abs': [],
                        'dc': [],
                        'pr': [],
                        'c_coeff': [],
                        'aspl': []
                        }

    changes_dict_maj = {'bc': [],
                        'cc': [],
                        'evc_norm': [],
                        'evc': [],
                        'evc_abs': [],
                        'dc': [],
                        'pr': [],
                        'c_coeff': [],
                        'aspl': []
                        }

    recommended_nodes_dict = {}

    edge_type_dict = {}

    frac_mins_list = []

    # overlaps = []
    
    # wtf = None

    # if salsa_new and not intervene:
    wtf = who_to_follow_rank(A=nx.adjacency_matrix(network_obj), njobs=-1)
    
    # elif intervene:
    # wtf = recomms
        
    

    for i in range(n):

        if network_file == '':
            g = network_obj
        else:
            g = read_pkl_graph(network_file)
       
        g = reindex_nodes(g, name_as_str=False)

        a = Analyzer(network=g, class_name=class_name, min_label=min_label, maj_label=maj_label)

        if remove_zero_degree_nodes:
            _ = a.remove_zero_degree()

        if not quick_test:
            if i == 0:
                min_c_coeff, maj_c_coeff = get_avg_clustering_coefficient(network_obj=a.network, mode='zero', class_name=class_name, group_names=[min_label, maj_label])
                min_aspl, maj_aspl = get_avg_sp_lengths(network_obj=a.network, mode='out', class_name=class_name, group_names=[min_label, maj_label])

            g_pre = nx_to_ig(a.network)

            
            _ = a.get_group_betweenness(g_pre, is_pre=True, verbose=verbose)
            _ = a.get_group_closeness(g_pre, is_pre=True, verbose=verbose)
            _ = a.get_group_eigenvector_centrality(g_pre, is_pre=True, verbose=verbose)
            _ = a.get_group_degree_centrality(g_pre, is_pre=True, verbose=verbose)
            _ = a.get_group_pagerank(g_pre, is_pre=True, verbose=verbose)

        if remove_zero_degree_nodes:
            if len(a.to_remove) > 0:
                a.add_nodes()
        
        # if not salsa_new:
        #     _ = a.sample_and_delete_edges(num_to_sample=num_to_sample, remove_edges=delete_edges)


        
        salsa_edges_per_node, salsa_all_edges, edge_type, avg_frac_min = a.reconstruct_edges(strategy=strategy, 
                                            salsa_new=salsa_new, run=i, wtf=wtf, to_pick=to_pick, consider_all_nodes=consider_all_nodes, 
                                            num_to_consider=num_to_consider, add_top_k=add_top_k, delete_edges=delete_edges,
                                            network_file=network_file, network_obj=network_obj, class_name=class_name,
                                            intervene=intervene, network=network_obj, strat=strat, metric_mode=metric_mode, j=j, target_mode=target_mode)
        

        edge_type_dict[i] = edge_type
        frac_mins_list.append(avg_frac_min)
        
        if remove_zero_degree_nodes:
            _ = a.remove_zero_degree()

        min_ccoeff_post, maj_ccoeff_post = get_avg_clustering_coefficient(network_obj=a.network, mode='zero', class_name=class_name, group_names=[min_label, maj_label])
        min_aspl_post, maj_aspl_post = get_avg_sp_lengths(network_obj=a.network, mode='out', class_name=class_name, group_names=[min_label, maj_label])

        changes_dict_maj['c_coeff'].append(np.round((maj_ccoeff_post/maj_c_coeff-1)*100, round))
        changes_dict_min['c_coeff'].append(np.round((min_ccoeff_post/min_c_coeff-1)*100, round))

        changes_dict_maj['aspl'].append(np.round((maj_aspl_post/maj_aspl-1)*100, round))
        changes_dict_min['aspl'].append(np.round((min_aspl_post/min_aspl-1)*100, round))

        g_post = nx_to_ig(a.network)

        _ = a.get_group_betweenness(g_post, is_pre=False, verbose=verbose)
        _ = a.get_group_closeness(g_post, is_pre=False, verbose=verbose)
        _ = a.get_group_eigenvector_centrality(g_post, is_pre=False, verbose=verbose)
        _ = a.get_group_degree_centrality(g_post, is_pre=False, verbose=verbose)
        _ = a.get_group_pagerank(g_post, is_pre=False, verbose=verbose)
        
        bc_vals = a.metrics_dict['bc']
        changes_dict_maj['bc'].append(np.round((bc_vals['post']['maj']/bc_vals['pre']['maj']-1)*100, round))
        changes_dict_min['bc'].append(np.round((bc_vals['post']['min']/bc_vals['pre']['min']-1)*100, round))

        cc_vals = a.metrics_dict['cc']
        changes_dict_maj['cc'].append(np.round((cc_vals['post']['maj']/cc_vals['pre']['maj']-1)*100, round))
        changes_dict_min['cc'].append(np.round((cc_vals['post']['min']/cc_vals['pre']['min']-1)*100, round))

        evc_vals = a.metrics_dict['evc']
        norm_evc_min_pre = evc_vals['pre']['min'] / (evc_vals['pre']['min'] + evc_vals['pre']['maj'])
        norm_evc_maj_pre = evc_vals['pre']['maj'] / (evc_vals['pre']['min'] + evc_vals['pre']['maj'])

        norm_evc_min_post = evc_vals['post']['min'] / (evc_vals['post']['min'] + evc_vals['post']['maj'])
        norm_evc_maj_post = evc_vals['post']['maj'] / (evc_vals['post']['min'] + evc_vals['post']['maj'])

        changes_dict_maj['evc_norm'].append(np.round((norm_evc_maj_post/norm_evc_maj_pre-1)*100, round))
        changes_dict_min['evc_norm'].append(np.round((norm_evc_min_post/norm_evc_min_pre-1)*100, round))

        changes_dict_maj['evc'].append(np.round((evc_vals['post']['maj']/evc_vals['pre']['maj']-1)*100, round))
        changes_dict_min['evc'].append(np.round((evc_vals['post']['min']/evc_vals['pre']['min']-1)*100, round))

        # changes_dict_maj['evc_abs'].append(np.round((, round))
        # changes_dict_min['evc_abs'].append(np.round((evc_vals['post']['min']/evc_vals['pre']['min']-1)*100, round))

        dc_vals = a.metrics_dict['dc']
        changes_dict_maj['dc'].append(np.round((dc_vals['post']['maj']/dc_vals['pre']['maj']-1)*100, round))
        changes_dict_min['dc'].append(np.round((dc_vals['post']['min']/dc_vals['pre']['min']-1)*100, round))

        pr_vals = a.metrics_dict['pr']
        changes_dict_maj['pr'].append(np.round((pr_vals['post']['maj']/pr_vals['pre']['maj']-1)*100, round))
        changes_dict_min['pr'].append(np.round((pr_vals['post']['min']/pr_vals['pre']['min']-1)*100, round))

        recommended_nodes_dict[i] = salsa_all_edges

        print(f'Finished run {i}')

    # return changes_dict_maj, changes_dict_min
    avg_bc_change_maj, sd_bc_maj = np.round(np.average(changes_dict_maj['bc']), round), np.round(np.std(changes_dict_maj['bc']), round)
    avg_bc_change_min, sd_bc_min = np.round(np.average(changes_dict_min['bc']), round), np.round(np.std(changes_dict_min['bc']), round)

    avg_cc_change_maj, sd_cc_maj = np.round(np.average(changes_dict_maj['cc']), round), np.round(np.std(changes_dict_maj['cc']), round)
    avg_cc_change_min, sd_cc_min = np.round(np.average(changes_dict_min['cc']), round), np.round(np.std(changes_dict_min['cc']), round)

    avg_evcn_change_maj, sd_evcn_maj = np.round(np.average(changes_dict_maj['evc_norm']), round), np.round(np.std(changes_dict_maj['evc_norm']), round)
    avg_evcn_change_min, sd_evcn_min = np.round(np.average(changes_dict_min['evc_norm']), round), np.round(np.std(changes_dict_min['evc_norm']), round)

    avg_evc_change_maj, sd_evc_maj = np.round(np.average(changes_dict_maj['evc']), round), np.round(np.std(changes_dict_maj['evc']), round)
    avg_evc_change_min, sd_evc_min = np.round(np.average(changes_dict_min['evc']), round), np.round(np.std(changes_dict_min['evc']), round)

    avg_dc_change_maj, sd_dc_maj = np.round(np.average(changes_dict_maj['dc']), round), np.round(np.std(changes_dict_maj['dc']), round)
    avg_dc_change_min, sd_dc_min = np.round(np.average(changes_dict_min['dc']), round), np.round(np.std(changes_dict_min['dc']), round)

    avg_pr_change_maj, sd_pr_maj = np.round(np.average(changes_dict_maj['pr']), round), np.round(np.std(changes_dict_maj['pr']), round)
    avg_pr_change_min, sd_pr_min = np.round(np.average(changes_dict_min['pr']), round), np.round(np.std(changes_dict_min['pr']), round)

    avg_ccoeff_change_maj, sd_ccoeff_maj = np.round(np.average(changes_dict_maj['c_coeff']), round), np.round(np.std(changes_dict_maj['c_coeff']), round)
    avg_ccoeff_change_min, sd_ccoeff_min = np.round(np.average(changes_dict_min['c_coeff']), round), np.round(np.std(changes_dict_min['c_coeff']), round)

    avg_aspl_change_maj, sd_aspl_maj = np.round(np.average(changes_dict_maj['aspl']), round), np.round(np.std(changes_dict_maj['aspl']), round)
    avg_aspl_change_min, sd_aspl_min = np.round(np.average(changes_dict_min['aspl']), round), np.round(np.std(changes_dict_min['aspl']), round)

    mm = []
    mM = []
    MM = []
    Mm = []

    if network_file == '':
            network = network_obj
    else:
            network = read_pkl_graph(network_file)

    for run in recommended_nodes_dict.keys():

        edge_type = {'mm': 0,
            'MM': 0,
            'mM': 0,
            'Mm': 0}

        for edge in recommended_nodes_dict[run]:
            class_k = network.nodes()[edge[0]][class_name]
            class_j = network.nodes()[edge[-1]][class_name]
            
            if class_k == 1 and class_j == 1:
                edge_type['mm'] += 1
            elif class_k == 1 and class_j == 0:
                edge_type['mM'] += 1
            elif class_k == 0 and class_j == 1:
                edge_type['Mm'] += 1
            elif class_k == 0 and class_j == 0:
                edge_type['MM'] += 1
        
 
        mm.append(edge_type['mm'] / len(recommended_nodes_dict[run]) * 100)
        MM.append(edge_type['MM'] / len(recommended_nodes_dict[run]) * 100)
        mM.append(edge_type['mM'] / len(recommended_nodes_dict[run]) * 100)
        Mm.append(edge_type['Mm'] / len(recommended_nodes_dict[run]) * 100)

    # for run in edge_type_dict.keys():
    #     mm.append(edge_type_dict[run]['mm'])
    #     mM.append(edge_type_dict[run]['mM'])
    #     MM.append(edge_type_dict[run]['MM'])
    #     Mm.append(edge_type_dict[run]['Mm'])

    # mm = [e/len(salsa_all_edges)*100 for e in mm]
    # mM = [e/len(salsa_all_edges)*100 for e in mM]
    # MM = [e/len(salsa_all_edges)*100 for e in MM]
    # Mm = [e/len(salsa_all_edges)*100 for e in Mm]

    avg_mm, sd_mm = np.round(np.average(mm), round), np.round(np.std(mm), round)
    avg_mM, sd_mM = np.round(np.average(mM), round), np.round(np.std(mM), round)
    avg_MM, sd_MM = np.round(np.average(MM), round), np.round(np.std(MM), round)
    avg_Mm, sd_Mm = np.round(np.average(Mm), round), np.round(np.std(Mm), round)

    # recommendations
    avg_avg_frac_min, sd_frac_min = [], []
    if strategy != 'random':
        avg_avg_frac_min, sd_frac_min = np.round(np.average(frac_mins_list), round), np.round(np.std(frac_mins_list), round)
    
    # additions
    avg_recomm_share_min, recomm_sd_min, avg_recomm_share_maj, recomm_sd_maj = get_recommendation_group_ratios(network_file='', network_obj=network_obj, recommended_nodes=recommended_nodes_dict, class_name=class_name)
    # avg_recomm_share_min, recomm_sd_min, avg_recomm_share_maj, recomm_sd_maj = 0, 0, 0, 0
    if to_txt:
        save_as_txt = save_as[:-4] + '.txt'
        f = open(save_as_txt, 'w')

        f.write("\n")
        f.write(f"Mean stats and standard deviation for {n} runs\n")
        f.write("---------------------------------------------------\n")
        f.write("\n")
        f.write('Added Nodes Min/Maj Fraction\n')
        f.write("---------------------------\n")
        f.write(f'Avg. share of maj: {avg_recomm_share_maj}% ({recomm_sd_maj})\n')
        f.write(f'Avg. share of min: {avg_recomm_share_min}% ({recomm_sd_min})\n')
        f.write("\n")
        f.write('Top 10 Minority Fraction Recommendations\n')
        f.write("---------------------------\n")
        f.write(f'Avg. minority fraction among Top 10 Recommendations: {avg_avg_frac_min}% ({sd_frac_min})\n')
        f.write("\n")
        f.write('Edge types\n')
        f.write("---------------------------\n")
        f.write(f'mm: {avg_mm}% ({sd_mm})\n')
        f.write(f'mM: {avg_mM}% ({sd_mM})\n')
        f.write(f'MM: {avg_MM}% ({sd_MM})\n')
        f.write(f'Mm: {avg_Mm}% ({sd_Mm})\n')
        f.write("\n")
        f.write('DC Changes\n')
        f.write("---------------------------\n")
        f.write(f"Majority: {avg_dc_change_maj} ({sd_dc_maj})\n")
        f.write(f"Minority: {avg_dc_change_min} ({sd_dc_min})\n")
        f.write("\n")
        f.write('BC Changes\n')
        f.write("---------------------------\n")
        f.write(f"Majority: {avg_bc_change_maj} ({sd_bc_maj})\n")
        f.write(f"Minority: {avg_bc_change_min} ({sd_bc_min})\n")
        f.write("\n")
        f.write('CC Changes\n')
        f.write("---------------------------\n")
        f.write(f"Majority: {avg_cc_change_maj} ({sd_cc_maj})\n")
        f.write(f"Minority: {avg_cc_change_min} ({sd_cc_min})\n")
        f.write("\n")
        f.write('EVC Changes\n')
        f.write("---------------------------\n")
        f.write(f"Majority: {avg_evc_change_maj} ({sd_evc_maj})\n")
        f.write(f"Minority: {avg_evc_change_min} ({sd_evc_min})\n")
        f.write("\n")
        f.write('Norm EVC Changes\n')
        f.write("---------------------------\n")
        f.write(f"Majority: {avg_evcn_change_maj} ({sd_evcn_maj})\n")
        f.write(f"Minority: {avg_evcn_change_min} ({sd_evcn_min})\n")
        f.write("\n")
        f.write('PR Changes\n')
        f.write("---------------------------\n")
        f.write(f"Majority: {avg_pr_change_maj} ({sd_pr_maj})\n")
        f.write(f"Minority: {avg_pr_change_min} ({sd_pr_min})\n")
        f.write("\n")
        f.write('Clustering Coeff Changes\n')
        f.write("---------------------------\n")
        f.write(f"Majority: {avg_ccoeff_change_maj} ({sd_ccoeff_maj})\n")
        f.write(f"Minority: {avg_ccoeff_change_min} ({sd_ccoeff_min})\n")
        f.write("\n")
        f.write('Avg. Shortest Path Changes\n')
        f.write("---------------------------\n")
        f.write(f"Majority: {avg_aspl_change_maj} ({sd_aspl_maj})\n")
        f.write(f"Minority: {avg_aspl_change_min} ({sd_aspl_min})\n")
        f.write("\n")
        # f.write(f"Avg. overlap: {np.round(np.average(overlaps), round)}\n")

        f.close()

    print("")
    print(f"Mean stats and standard deviation for {n} runs")
    print("---------------------------------------------------")
    print("")
    print('Added Nodes Min/Maj Fraction')
    print("---------------------------")
    print(f'Avg. share of maj: {avg_recomm_share_maj}% ({recomm_sd_maj})')
    print(f'Avg. share of min: {avg_recomm_share_min}% ({recomm_sd_min})')
    print("")
    print('Edge types')
    print("---------------------------")
    print(f'mm: {avg_mm}% ({sd_mm})')
    print(f'mM: {avg_mM}% ({sd_mM})')
    print(f'MM: {avg_MM}% ({sd_MM})')
    print(f'Mm: {avg_Mm}% ({sd_Mm})')
    print("")
    print('Top 10 Minority Fraction Recommendations')
    print("---------------------------")
    print(f'Avg. minority fraction: {avg_avg_frac_min}% ({sd_frac_min})')
    print("")
    print('DC Changes')
    print("---------------------------")
    print(f"Majority: {avg_dc_change_maj} ({sd_dc_maj})")
    print(f"Minority: {avg_dc_change_min} ({sd_dc_min})")
    print("")
    print('BC Changes')
    print("---------------------------")
    print(f"Majority: {avg_bc_change_maj} ({sd_bc_maj})")
    print(f"Minority: {avg_bc_change_min} ({sd_bc_min})")
    print("")
    print('CC Changes')
    print("---------------------------")
    print(f"Majority: {avg_cc_change_maj} ({sd_cc_maj})")
    print(f"Minority: {avg_cc_change_min} ({sd_cc_min})")
    print("")
    print('EVC Changes')
    print("---------------------------")
    print(f"Majority: {avg_evc_change_maj} ({sd_evc_maj})")
    print(f"Minority: {avg_evc_change_min} ({sd_evc_min})")
    print("")
    print('Norm EVC Changes')
    print("---------------------------")
    print(f"Majority: {avg_evcn_change_maj} ({sd_evcn_maj})")
    print(f"Minority: {avg_evcn_change_min} ({sd_evcn_min})")
    print("")
    print('PR Changes')
    print("---------------------------")
    print(f"Majority: {avg_pr_change_maj} ({sd_pr_maj})")
    print(f"Minority: {avg_pr_change_min} ({sd_pr_min})")
    print("")
    print('Clustering Coeff Changes')
    print("---------------------------")
    print(f"Majority: {avg_ccoeff_change_maj} ({sd_ccoeff_maj})")
    print(f"Minority: {avg_ccoeff_change_min} ({sd_ccoeff_min})")
    print("")
    print('Avg. Shortest Path Changes')
    print("---------------------------")
    print(f"Majority: {avg_aspl_change_maj} ({sd_aspl_maj})")
    print(f"Minority: {avg_aspl_change_min} ({sd_aspl_min})")
    print("")
    # print(f"Avg. overlap: {np.round(np.average(overlaps), round)}")

    results = {'pct_changes': {'min': changes_dict_min, 'maj': changes_dict_maj}, 
               'frac_mins': [frac_mins_list, sd_frac_min], 'edge_types': edge_type_dict, 
               'recommended_nodes': recommended_nodes_dict}
    if save_results:
        
        with open(save_as, 'wb') as f:
            pickle.dump(results, f)
        print('')
        print(f'Results saved to disk as {save_as}')

    return results

            

In [12]:
def get_recommendation_group_ratios(network_file='', network_obj=None, recommended_nodes=[], class_name='m'):
    
    if network_file == '':
        network = network_obj
    else:
        network = read_pkl_graph(network_file)

    all_nodes = network.nodes()
    shares_min = []
    shares_maj = []


    for i in recommended_nodes.keys():

        count_maj = 0
        count_min = 0
       

        for tup in recommended_nodes[i]:
            node = tup[-1]
            if all_nodes[node][class_name] == 0:
                count_maj += 1
            elif all_nodes[node][class_name] == 1:
                count_min += 1

        shares_min.append(count_min/len(recommended_nodes[i])*100)
        shares_maj.append(count_maj/len(recommended_nodes[i])*100)

    avg_recomm_share_min, recomm_sd_min = np.round(np.average(shares_min), 5), np.round(np.std(shares_min), 5)
    avg_recomm_share_maj, recomm_sd_maj = np.round(np.average(shares_maj), 5), np.round(np.std(shares_maj), 5)

    
    return avg_recomm_share_min, recomm_sd_min, avg_recomm_share_maj, recomm_sd_maj

In [13]:
def get_avg_clustering_coefficient(network_file='', network_obj=None, mode='zero', class_name='m', group_names=[1, 0], verbose=False):

    def get_nodes_per_class(network, class_name='m', group_names=[1, 0]):
        
        # class_
        if class_name == 'm':
            min_nodes_ig = network.vs.select(m=group_names[0])
            maj_nodes_ig = network.vs.select(m=group_names[-1])

        elif class_name == 'class_':
            min_nodes_ig = network.vs.select(class_=group_names[0])
            maj_nodes_ig = network.vs.select(class_=group_names[-1])

        elif class_name == 'age':
            min_nodes_ig = network.vs.select(age=group_names[0])
            maj_nodes_ig = network.vs.select(age=group_names[-1])

        return min_nodes_ig, maj_nodes_ig
    
    if network_file == '':
        network = network_obj
    else:
        network = read_pkl_graph(network_file)

    network_ig = nx_to_ig(network)

    min_nodes, maj_nodes = get_nodes_per_class(network_ig, class_name=class_name, group_names=group_names)

    # mode: how to treat nodes with degree < 2
    transitivity = network_ig.transitivity_local_undirected(mode=mode)
    network_ig.vs['clustering_coeff'] = transitivity

    min_clustering_coeff = np.average(np.array(min_nodes.get_attribute_values('clustering_coeff')))
    maj_clustering_coeff = np.average(np.array(maj_nodes.get_attribute_values('clustering_coeff')))

    if verbose:
        print(f'Min clustering coeff: {min_clustering_coeff}')
        print(f'Maj clustering coeff: {maj_clustering_coeff}')

    return min_clustering_coeff, maj_clustering_coeff
    

In [14]:
def get_avg_sp_lengths(network_file='', network_obj=None, mode='out', class_name='m', group_names=[1, 0], verbose=False):

    def get_nodes_per_class(network, class_name='m', group_names=['young', 'old']):
        
        # class_
        if class_name == 'm':
            min_nodes_ig = network.vs.select(m=group_names[0])
            maj_nodes_ig = network.vs.select(m=group_names[-1])

        elif class_name == 'class_':
            min_nodes_ig = network.vs.select(class_=group_names[0])
            maj_nodes_ig = network.vs.select(class_=group_names[-1])

        elif class_name == 'age':
            min_nodes_ig = network.vs.select(age=group_names[0])
            maj_nodes_ig = network.vs.select(age=group_names[-1])

        return min_nodes_ig, maj_nodes_ig
    
    if network_file == '':
        network = network_obj
    else:
        network = read_pkl_graph(network_file)
    network_ig = nx_to_ig(network)

    shortest_path_lengths = network_ig.shortest_paths(mode=mode)
    arr = np.array(shortest_path_lengths)
    arr[arr==inf] = 0
    avg_sp_lens = [np.average(lens) for lens in arr]

    min_nodes, maj_nodes = get_nodes_per_class(network_ig, class_name=class_name, group_names=group_names)

    network_ig.vs['avg_sp_len'] = avg_sp_lens

    min_avg_sp = np.average(np.array(min_nodes.get_attribute_values('avg_sp_len')))
    maj_avg_sp = np.average(np.array(maj_nodes.get_attribute_values('avg_sp_len')))

    if verbose:
        print(f'Min avg sp length: {min_avg_sp}')
        print(f'Maj avg sp length: {maj_avg_sp}')

    return min_avg_sp, maj_avg_sp


In [15]:
def plot_degree_dist_per_group(G, attribute='', group_names=['', ''], group_labels=[1, 0]):
    minority = [G.degree(n[0]) for n in G.nodes(data=True) if n[-1][attribute] == group_labels[0]]
    majority = [G.degree(n[0]) for n in G.nodes(data=True) if n[-1][attribute] == group_labels[1]]
    
    fig, ax = plt.subplots(1, 2, sharey=True)
    fig.suptitle('Degree distribution by group')
    
    sns.histplot(ax=ax[0], data=minority)
    # ax[0].set_ylabel('test')
    ax[0].set_xlabel('Degree')
    ax[0].set_title(f'{group_names[0]} (minority)')

    sns.histplot(ax=ax[1], data=majority)
    ax[1].set_xlabel('Degree')
    ax[1].set_title(f'{group_names[1]} (majority)')

    plt.show()

In [16]:
def get_group_ratios(network_obj, attribute='', group_names=['', ''], group_labels=[1, 0]):

    minority = [n for n in network_obj.nodes(data=True) if n[-1][attribute] == group_labels[0]]
    majority = [n for n in network_obj.nodes(data=True) if n[-1][attribute] == group_labels[1]]

    print(f'# {group_names[0]} nodes (minority): {len(minority)} ({np.round(len(minority)/(len(minority)+len(majority))*100, 2)}%)')
    print(f'# {group_names[1]} nodes (majority): {len(majority)} ({np.round(len(majority)/(len(minority)+len(majority))*100, 2)}%)')

In [17]:
def plot_degree_dist(G):
    degrees = [G.degree(n) for n in G.nodes()]
    sns.histplot(degrees)
    plt.xlabel('Degree')
    plt.show()

In [18]:
def enum_vnum(network):
    print(f'Number of edges: {network.number_of_edges()}')
    print(f'Number of nodes: {network.number_of_nodes()}')

In [19]:
def reindex_nodes(network, name_as_str=True):

    old = list(network.nodes())
    new = list(range(0, len(old)))

    if name_as_str:
        old_str = [str(i) for i in old]
        mapping1 = dict(zip(old, old_str))
    else:
        mapping1 = dict(zip(old, old))
    nx.set_node_attributes(network, values=mapping1, name='name')

    mapping2 = dict(zip(old, new))

    return nx.relabel_nodes(network, mapping2, copy=True)

In [20]:
def hubs_auths_analysis(network_obj, ha_dict):

    mins_hubs = []
    mins_auths = []

    majs_hubs = []
    majs_auths = []

    for i, dic in enumerate(ha_dict):
        node_cls = network_obj.nodes()[i]['m']
        
        hubs_cls = [network_obj.nodes()[hub]['m'] for hub in dic[i]['hubs']]
        frac_min_hubs = sum(hubs_cls) / len(hubs_cls)

        auths_cls = [network_obj.nodes()[auth]['m'] for auth in dic[i]['auths']]
        frac_min_auths = sum(auths_cls) / len(auths_cls)

        if node_cls == 1:
            mins_hubs.append(frac_min_hubs)
            mins_auths.append(frac_min_auths)
        else:
            majs_hubs.append(frac_min_hubs)
            majs_auths.append(frac_min_auths)

    print(f'Avg. frac. min. among hubs if node is minority: {np.round(np.average(mins_hubs), 3)}')
    print(f'Avg. frac. min. among authorities if node is minority: {np.round(np.average(mins_auths), 3)}')

    print(f'Avg. frac. min. among hubs if node is majority: {np.round(np.average(majs_hubs), 3)}')
    print(f'Avg. frac. min. among authorities if node is majority: {np.round(np.average(majs_auths), 3)}')

In [30]:
def recommendation_analysis(wtf_scores, network):
    in_degs = []
    for arr in wtf_scores:
        in_deg = []
        for node in arr:
            in_deg.append(network.in_degree(node))
        in_degs.append(in_deg)
    
    avg_med_indegs = [(np.average(arr), np.median(arr)) for arr in in_degs]

    total_avg = np.average([tup[0] for tup in avg_med_indegs])
    total_median = np.median([tup[1] for tup in avg_med_indegs])

    all_indegs = [network.in_degree(node) for node in network.nodes()]

    avg_all_indegs = np.average(all_indegs)
    median_all_indegs = np.median(all_indegs)

    print('Recommendations in-degree')
    print('---------------------------')
    print(f'Avg: {total_avg}')
    print(f'Median: {total_median}')
    print('')
    print('Overall network in-degree')
    print('---------------------------')
    print(f'Avg: {avg_all_indegs}')
    print(f'Median: {median_all_indegs}')

    return {'recomms': (total_avg, total_median), 'network': (avg_all_indegs, median_all_indegs)}

#### synthetic data

##### n82

In [203]:
network82 = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0802.pkl')
network82 = reindex_nodes(network82)
wtf_scores = who_to_follow_rank(A=nx.adjacency_matrix(network82), njobs=-1)
a82 = recommendation_analysis(wtf_scores, network82)

In [ ]:
network82 = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0802.pkl')
network82 = reindex_nodes(network82)
ha_dict = who_to_follow_rank(A=nx.adjacency_matrix(network82), njobs=-1)

hubs_auths_analysis(network82, ha_dict)

In [ ]:
# base

network82_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0802.pkl')
network82_topk = reindex_nodes(network82_topk)
network82_results_2000nodes_final = run_n_simulations(network_obj=network82_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='nx82_2000nodes_final.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# alternative strategy

network82_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0802.pkl')
network82_topk = reindex_nodes(network82_topk)
network82_results_2000nodes_new = run_n_simulations(network_obj=network82_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='new', save_results=True, save_as='nx82_2000nodes_new.pkl', 
                                            to_txt=True, salsa_new=False, quick_test=False, to_pick=5, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# intervention #1

network82_topk = nx.read_gpickle(config.ROOT + "nx_DPAH_10000_0208_0802.pkl")
network82_topk = reindex_nodes(network82_topk)

network82_results_2000nodes = run_n_simulations(network_obj=network82_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='intervention', save_results=True, save_as='82_base_ix.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# intervention #2

network82_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0802.pkl')
network82_topk = reindex_nodes(network82_topk)

top_maj_evc, top_min_evc = get_topj_evc_nodes(network_obj=network82_topk, j=10, to_ig=True, mode='both')
results = get_intervention_recommendations(A=nx.adjacency_matrix(network82_topk), to_add=top_maj_evc, p=0.85, top=10, num_cores=-1, network_obj=None)

network82_results_2000nodes = run_n_simulations(network_obj=network82_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='82_new_ix.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True,
                                            recomms=results)

##### n88

In [ ]:
network88 = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0808.pkl')
network88 = reindex_nodes(network88)
wtf_scores_88 = who_to_follow_rank(A=nx.adjacency_matrix(network88), njobs=-1)

a88 = recommendation_analysis(wtf_scores_88, network88)

In [ ]:
network88 = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0808.pkl')
network88 = reindex_nodes(network88)
ha_dict = who_to_follow_rank(A=nx.adjacency_matrix(network88), njobs=-1)

hubs_auths_analysis(network88, ha_dict)

In [ ]:
# base

network88_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0808.pkl')
network88_topk = reindex_nodes(network88_topk)
network88_results_2000nodes_final = run_n_simulations(network_obj=network88_topk, n=10, 
                                            num_to_sample=2000, delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='nx88_2000nodes_final.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# alternative strategy

network88_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0808.pkl')
network88_topk = reindex_nodes(network88_topk)
network88_results_2000nodes_new = run_n_simulations(network_obj=network88_topk, n=10, 
                                            num_to_sample=2000, delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='new', save_results=True, save_as='nx88_2000nodes_new.pkl', 
                                            to_txt=True, salsa_new=False, quick_test=False, to_pick=5, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# intervention #1

network88_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0808.pkl')
network88_topk = reindex_nodes(network88_topk)

network88_results_2000nodes = run_n_simulations(network_obj=network88_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='intervention', save_results=True, save_as='88_base_ix.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# intervention #2

network88_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0808.pkl')
network88_topk = reindex_nodes(network88_topk)

top_maj_evc, top_min_evc = get_topj_evc_nodes(network_obj=network88_topk, j=10, to_ig=True, mode='both')
results = get_intervention_recommendations(A=nx.adjacency_matrix(network88_topk), to_add=top_maj_evc, p=0.85, top=10, num_cores=-1, network_obj=None)

network88_results_2000nodes = run_n_simulations(network_obj=network88_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='88_new_ix.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True,
                                            recomms=results)

##### n28

In [ ]:
network28 = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0208.pkl')
network28 = reindex_nodes(network28)
wtf_scores_28 = who_to_follow_rank(A=nx.adjacency_matrix(network28), njobs=-1)

a28 = recommendation_analysis(wtf_scores_28, network28)

In [ ]:
network28 = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0208.pkl')
network28 = reindex_nodes(network28)
ha_dict = who_to_follow_rank(A=nx.adjacency_matrix(network28), njobs=-1)

hubs_auths_analysis(network28, ha_dict)

In [ ]:
# base

network28_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0208.pkl')
network28_topk = reindex_nodes(network28_topk)
network28_results_2000nodes_final = run_n_simulations(network_obj=network28_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='nx28_2000nodes_final.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# alternative strategy

network28_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0208.pkl')
network28_topk = reindex_nodes(network28_topk)
network28_results_2000nodes_new = run_n_simulations(network_obj=network28_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='new', save_results=True, save_as='nx28_2000nodes_new.pkl', 
                                            to_txt=True, salsa_new=False, quick_test=False, to_pick=5, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# intervention #1

network28_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0208.pkl')
network28_topk = reindex_nodes(network28_topk)

network28_results_2000nodes = run_n_simulations(network_obj=network28_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='intervention', save_results=True, save_as='28_base_ix.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# intervention #2

network28_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0208.pkl')
network28_topk = reindex_nodes(network28_topk)

top_maj_evc, top_min_evc = get_topj_evc_nodes(network_obj=network28_topk, j=10, to_ig=True, mode='both')
results = get_intervention_recommendations(A=nx.adjacency_matrix(network28_topk), to_add=top_maj_evc, p=0.85, top=10, num_cores=-1, network_obj=None)

network28_results_2000nodes = run_n_simulations(network_obj=network28_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='28_new_ix.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True,
                                            recomms=results)

##### n22

In [ ]:
network22 = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0202.pkl')
network22 = reindex_nodes(network22)
wtf_scores_22 = who_to_follow_rank(A=nx.adjacency_matrix(network22), njobs=-1)

a22 = recommendation_analysis(wtf_scores_22, network22)

In [ ]:
network22 = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0202.pkl')
network22 = reindex_nodes(network22)
ha_dict = who_to_follow_rank(A=nx.adjacency_matrix(network22), njobs=-1)

hubs_auths_analysis(network22, ha_dict)

In [ ]:
# base

network22_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0202.pkl')
network22_topk = reindex_nodes(network22_topk)
network22_results_2000nodes_final = run_n_simulations(network_obj=network22_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='nx22_2000nodes_final.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# alternative strategy

network22_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0202.pkl')
network22_topk = reindex_nodes(network22_topk)
network22_results_2000nodes_new = run_n_simulations(network_obj=network22_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='new', save_results=True, save_as='nx22_2000nodes_new.pkl', 
                                            to_txt=True, salsa_new=False, quick_test=False, to_pick=5, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# intervention #1

network22_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0202.pkl')
network22_topk = reindex_nodes(network22_topk)

network22_results_2000nodes = run_n_simulations(network_obj=network22_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='intervention', save_results=True, save_as='22_base_ix.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# intervention #2

network22_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0202.pkl')
network22_topk = reindex_nodes(network22_topk)

top_maj_evc, top_min_evc = get_topj_evc_nodes(network_obj=network22_topk, j=10, to_ig=True, mode='both')
results = get_intervention_recommendations(A=nx.adjacency_matrix(network22_topk), to_add=top_maj_evc, p=0.85, top=10, num_cores=-1, network_obj=None)

network22_results_2000nodes = run_n_simulations(network_obj=network22_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='22_new_ix.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True,
                                            recomms=results)

##### n55

In [ ]:
network55 = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0505.pkl')
network55 = reindex_nodes(network55)
wtf_scores_55 = who_to_follow_rank(A=nx.adjacency_matrix(network55), njobs=-1)

a55 = recommendation_analysis(wtf_scores_55, network55)

In [ ]:
network55 = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0505.pkl')
network55 = reindex_nodes(network55)
ha_dict = who_to_follow_rank(A=nx.adjacency_matrix(network55), njobs=-1)

hubs_auths_analysis(network55, ha_dict)

In [ ]:
# base

network55_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0505.pkl')
network55_topk = reindex_nodes(network55_topk)
network55_results_2000nodes_final = run_n_simulations(network_obj=network55_topk, n=10, 
                                            elete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='nx55_2000nodes_final.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# alternative strategy

network55_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0505.pkl')
network55_topk = reindex_nodes(network55_topk)
network55_results_2000nodes_new = run_n_simulations(network_obj=network55_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='new', save_results=True, save_as='nx55_2000nodes_new.pkl', 
                                            to_txt=True, salsa_new=False, quick_test=False, to_pick=5, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# intervention #1

network55_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0505.pkl')
network55_topk = reindex_nodes(network55_topk)

network55_results_2000nodes = run_n_simulations(network_obj=network55_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='intervention', save_results=True, save_as='55_base_ix.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True)

In [ ]:
# intervention #2

network55_topk = nx.read_gpickle(config.ROOT + 'nx_DPAH_10000_0208_0505.pkl')
network55_topk = reindex_nodes(network55_topk)

top_maj_evc, top_min_evc = get_topj_evc_nodes(network_obj=network55_topk, j=10, to_ig=True, mode='both')
results = get_intervention_recommendations(A=nx.adjacency_matrix(network55_topk), to_add=top_maj_evc, p=0.85, top=10, num_cores=-1, network_obj=None)

network55_results_2000nodes = run_n_simulations(network_obj=network55_topk, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='55_new_ix.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=2000, add_top_k=True,
                                            recomms=results)

#### Blogs network analysis

In [3]:
blogs = nx.read_gpickle(config.ROOT + "blogs_attributed_network_anon.gpickle")
blogs.number_of_edges()

19025

In [88]:
blogs = nx.read_gpickle(config.ROOT + "blogs_attributed_network_anon.gpickle")
blogs = reindex_nodes(blogs)
ha_dict = who_to_follow_rank(A=nx.adjacency_matrix(blogs), njobs=-1)

hubs_auths_analysis(blogs, ha_dict)

Avg. frac. min. among hubs if node is minority: 0.866
Avg. frac. min. among authorities if node is minority: 0.728
Avg. frac. min. among hubs if node is majority: 0.256
Avg. frac. min. among authorities if node is majority: 0.316


In [ ]:
# base

blogs = nx.read_gpickle(config.ROOT + "blogs_attributed_network_anon.gpickle")
blogs = reindex_nodes(blogs)

blogs_results = run_n_simulations(network_obj=blogs, n=10, 
                                            num_to_sample=2000, delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='blogs_new.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=220, add_top_k=True,
                                            intervene=False, recomms=None
                                            )

In [ ]:
# intervention #1

blogs = nx.read_gpickle(config.ROOT + "blogs_attributed_network_anon.gpickle")
blogs = reindex_nodes(blogs)

blogs_results = run_n_simulations(network_obj=blogs, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='intervention', save_results=True, save_as='blogs_new.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=220, add_top_k=True
                                            )

In [189]:
# intervention #2

blogs = nx.read_gpickle(config.ROOT + "blogs_attributed_network_anon.gpickle")
blogs = reindex_nodes(blogs)

top_maj_evc, top_min_evc = get_topj_evc_nodes(network_obj=blogs, j=10, to_ig=True, mode='both')
results = get_intervention_recommendations(A=nx.adjacency_matrix(blogs), to_add=[top_min_evc, top_maj_evc], p=0.85, top=10, num_cores=-1, network_obj=blogs)

blogs = nx.read_gpickle(config.ROOT + "blogs_attributed_network_anon.gpickle")
blogs = reindex_nodes(blogs)
blogs_results_ix = run_n_simulations(network_obj=blogs, n=10, 
                                            num_to_sample=2000, delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='blogs_ix_evc_both.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=220, add_top_k=True,
                                            recomms=results
                                            )

#### Hate network analysis

In [4]:
hate = nx.read_gpickle(config.ROOT + "hate_attributed_network_anon.gpickle")
hate.number_of_edges()

15141

In [89]:
hate = nx.read_gpickle(config.ROOT + "hate_attributed_network_anon.gpickle")
hate = reindex_nodes(hate)
ha_dict = who_to_follow_rank(A=nx.adjacency_matrix(hate), njobs=-1)

hubs_auths_analysis(hate, ha_dict)

Avg. frac. min. among hubs if node is minority: 0.513
Avg. frac. min. among authorities if node is minority: 0.558
Avg. frac. min. among hubs if node is majority: 0.139
Avg. frac. min. among authorities if node is majority: 0.236


In [ ]:
# base

hate = nx.read_gpickle(config.ROOT + "hate_attributed_network_anon.gpickle")
hate = reindex_nodes(hate)

hate_results = run_n_simulations(network_obj=hate, n=10, 
                                            num_to_sample=2000, delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='hate_new.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=1000, add_top_k=True,
                                            intervene=False, recomms=None
                                            )

In [ ]:
# intervention #1

hate = nx.read_gpickle(config.ROOT + "hate_attributed_network_anon.gpickle")
hate = reindex_nodes(hate)

hate_results = run_n_simulations(network_obj=hate, n=10, 
                                            delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='intervention', save_results=True, save_as='hate_new.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=1000, add_top_k=True
                                            )

In [200]:
# intervention #2

hate = nx.read_gpickle(config.ROOT + "hate_attributed_network_anon.gpickle")
hate = reindex_nodes(hate)

top_maj_evc, top_min_evc = get_topj_evc_nodes(network_obj=hate, j=10, to_ig=True, mode='both')
results = get_intervention_recommendations(A=nx.adjacency_matrix(hate), to_add=[top_min_evc, top_maj_evc], p=0.85, top=10, num_cores=-1, network_obj=hate)

hate = nx.read_gpickle(config.ROOT + "hate_attributed_network_anon.gpickle")
hate = reindex_nodes(hate)
hate_results_ix = run_n_simulations(network_obj=hate, n=10, 
                                            num_to_sample=2000, delete_edges=True, verbose=False, round=5, 
                                            remove_zero_degree_nodes=True, class_name='m', min_label=1, maj_label=0,
                                            strategy='consecutively', save_results=True, save_as='hate_ix_evc_both.pkl', 
                                            to_txt=True, salsa_new=True, quick_test=False, to_pick=3, 
                                            consider_all_nodes=False, num_to_consider=1000, add_top_k=True,
                                            recomms=results
                                            )